# Bigmart Sales Data Set

Retail is another industry which extensively uses analytics to optimize business processes. Tasks like product placement, inventory management, customized offers, product bundling, etc. are being smartly handled using data science techniques. As the name suggests, this data comprises of transaction records of a sales store. This is a regression problem. The data has 8523 rows of 12 variables.

Problem: Predict the sales of a store.

Tutorial : https://www.analyticsvidhya.com/blog/2016/02/bigmart-sales-solution-top-20/

Get Data : https://datahack.analyticsvidhya.com/contest/practice-problem-big-mart-sales-iii/


The data scientists at BigMart have collected 2013 sales data for 1559 products across 10 stores in different cities. Also, certain attributes of each product and store have been defined. The aim is to build a predictive model and find out the sales of each product at a particular store.

Using this model, BigMart will try to understand the properties of products and stores which play a key role in increasing sales.

In [1]:
# imports
import pandas
import numpy
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
# Read Data
dataset=pandas.read_csv('Data/Train_BigMart.csv')

#### Summarize the Dataset

In [3]:
# 1. Dimensions of Dataset
print(dataset.shape)

(8523, 12)


In [4]:
# 2. Look at the data itself.
display(dataset.head(10))

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350


#### Variable Description

Item_Identifier - Unique product ID

Item_Weight - Weight of product

Item_Fat_Content - Whether the product is low fat or not

Item_Visibility - The % of total display area of all products in a store allocated to the particular product

Item_Type - The category to which the product belongs

Item_MRP - Maximum Retail Price (list price) of the product

Outlet_Identifier - Unique store ID

Outlet_Establishment_Year - The year in which store was established

Outlet_Size - The size of the store in terms of ground area covered

Outlet_Location_Type - The type of city in which the store is located

Outlet_Type - Whether the outlet is just a grocery store or some sort of supermarket

Item_Outlet_Sales - Sales of the product in the particulat store. This is the outcome variable to be predicted.

 

In [5]:
# 3. Statistical Summary
display(dataset.describe()) # it will identify numeric data automatically

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [6]:
# 4. Check Missing Values
dataset.apply(lambda x: sum(x.isnull()),axis=0) # axis 0 means column wise. 1 means row wise.

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

We have 1463 missing values in Item_Weight and 2410 missing values in Outlet_Size

In [7]:
dataset[dataset['Item_Weight'].isnull()]
# 7th , 18th, 21st, 23rd , 29th (... so on) rows has Item_Weight NaN. 
# So we will go through data and check if there is any way to find out Item's weight or not.

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
18,DRI11,NaN,Low Fat,0.034238,Hard Drinks,113.2834,OUT027,1985,Medium,Tier 3,Supermarket Type3,2303.6680
21,FDW12,NaN,Regular,0.035400,Baking Goods,144.5444,OUT027,1985,Medium,Tier 3,Supermarket Type3,4064.0432
23,FDC37,NaN,Low Fat,0.057557,Baking Goods,107.6938,OUT019,1985,Small,Tier 1,Grocery Store,214.3876
29,FDC14,NaN,Regular,0.072222,Canned,43.6454,OUT019,1985,Small,Tier 1,Grocery Store,125.8362
36,FDV20,NaN,Regular,0.059512,Fruits and Vegetables,128.0678,OUT027,1985,Medium,Tier 3,Supermarket Type3,2797.6916
38,FDX10,NaN,Regular,0.123111,Snack Foods,36.9874,OUT027,1985,Medium,Tier 3,Supermarket Type3,388.1614
39,FDB34,NaN,Low Fat,0.026481,Snack Foods,87.6198,OUT027,1985,Medium,Tier 3,Supermarket Type3,2180.4950
49,FDS02,NaN,Regular,0.255395,Dairy,196.8794,OUT019,1985,Small,Tier 1,Grocery Store,780.3176
59,FDI26,NaN,Low Fat,0.061082,Canned,180.0344,OUT019,1985,Small,Tier 1,Grocery Store,892.1720


In [8]:
dataset
# looking at 7th , 18th, 21st, 23rd , 29th (and so on) items, 
# it seems that we can find out the Item_Weight from Item_Identifier

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350


In [9]:
# so lets cross check it. Lets check weather perticular Item_Identifier has same Item_Weight through out
dataset[dataset['Item_Identifier']=='FDW12']

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
21,FDW12,NaN,Regular,0.035400,Baking Goods,144.5444,OUT027,1985,Medium,Tier 3,Supermarket Type3,4064.0432
1788,FDW12,8.315,Regular,0.059541,Baking Goods,143.6444,OUT010,1998,NaN,Tier 3,Grocery Store,145.1444
2973,FDW12,8.315,reg,0.035627,Baking Goods,146.6444,OUT049,1999,Medium,Tier 1,Supermarket Type1,870.8664
5319,FDW12,8.315,Regular,0.035543,Baking Goods,143.3444,OUT013,1987,High,Tier 3,Supermarket Type1,1596.5884
6285,FDW12,8.315,Regular,0.035572,Baking Goods,144.4444,OUT046,1997,Small,Tier 1,Supermarket Type1,2902.8880
7004,FDW12,8.315,Regular,0.035644,Baking Goods,147.1444,OUT045,2002,NaN,Tier 2,Supermarket Type1,870.8664
7259,FDW12,8.315,Regular,0.035773,Baking Goods,144.3444,OUT017,2007,NaN,Tier 2,Supermarket Type1,3483.4656


In [10]:
# cross check
dataset[dataset['Item_Identifier']=='FDC37']

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
23,FDC37,NaN,Low Fat,0.057557,Baking Goods,107.6938,OUT019,1985,Small,Tier 1,Grocery Store,214.3876
4652,FDC37,15.5,Low Fat,0.032846,Baking Goods,106.7938,OUT013,1987,High,Tier 3,Supermarket Type1,3215.8140
6212,FDC37,15.5,Low Fat,0.032924,Baking Goods,107.2938,OUT049,1999,Medium,Tier 1,Supermarket Type1,2143.8760


In [11]:
# cross check
dataset[dataset['Item_Identifier']=='FDC14']

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
29,FDC14,NaN,Regular,0.072222,Canned,43.6454,OUT019,1985,Small,Tier 1,Grocery Store,125.8362
3609,FDC14,14.5,Regular,0.041482,Canned,40.0454,OUT017,2007,NaN,Tier 2,Supermarket Type1,209.7270
4297,FDC14,14.5,Regular,0.069042,Canned,41.4454,OUT010,1998,NaN,Tier 3,Grocery Store,41.9454
5760,FDC14,14.5,Regular,0.000000,Canned,41.0454,OUT018,2009,Medium,Tier 3,Supermarket Type2,545.2902
6257,FDC14,NaN,Regular,0.041049,Canned,41.2454,OUT027,1985,Medium,Tier 3,Supermarket Type3,1342.2528
6346,FDC14,14.5,Regular,0.041215,Canned,42.0454,OUT013,1987,High,Tier 3,Supermarket Type1,629.1810
6478,FDC14,14.5,Regular,0.041249,Canned,40.5454,OUT046,1997,Small,Tier 1,Supermarket Type1,1216.4166
7031,FDC14,14.5,Regular,0.041313,Canned,42.0454,OUT049,1999,Medium,Tier 1,Supermarket Type1,377.5086


In [12]:
item_avg_weight = dataset.pivot_table(values='Item_Weight', index='Item_Identifier')

In [13]:
# Lets cross check weather FDC14 has value 14.5 or not
item_avg_weight.T['FDC14']

Item_Weight    14.5
Name: FDC14, dtype: float64

In [14]:
item_avg_weight.head()
# Lets join this item_weights to the actual dataset item_weights by Item_Identifier

,Item_Weight
Item_Identifier,
DRA12,11.600
DRA24,19.350
DRA59,8.270
DRB01,7.390
DRB13,6.115


In [15]:
dataset=pandas.merge(dataset,item_avg_weight,on='Item_Identifier',how='left')

In [16]:
dataset = dataset.drop(columns="Item_Weight_x")
dataset = dataset.rename(columns={"Item_Weight_y": "Item_Weight"})

In [17]:
dataset.head()

,Item_Identifier,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Weight
0,FDA15,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,9.30
1,DRC01,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,5.92
2,FDN15,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,17.50
3,FDX07,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800,19.20
4,NCD19,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,8.93


In [18]:
#Check Missing Values
dataset.apply(lambda x: sum(x.isnull()),axis=0) # axis 0 means column wise. 1 means row wise.

Item_Identifier                 0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
Item_Weight                     4
dtype: int64

In [19]:
dataset[dataset['Item_Weight'].isnull()]
# still we have some null values

,Item_Identifier,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Weight
927,FDN52,Regular,0.130933,Frozen Foods,86.9198,OUT027,1985,Medium,Tier 3,Supermarket Type3,1569.9564,NaN
1922,FDK57,Low Fat,0.079904,Snack Foods,120.0440,OUT027,1985,Medium,Tier 3,Supermarket Type3,4434.2280,NaN
4187,FDE52,Regular,0.029742,Dairy,88.9514,OUT027,1985,Medium,Tier 3,Supermarket Type3,3453.5046,NaN
5022,FDQ60,Regular,0.191501,Baking Goods,121.2098,OUT019,1985,Small,Tier 1,Grocery Store,120.5098,NaN


In [20]:
dataset[dataset['Item_Identifier']=='FDN52']

,Item_Identifier,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Weight
927,FDN52,Regular,0.130933,Frozen Foods,86.9198,OUT027,1985,Medium,Tier 3,Supermarket Type3,1569.9564,NaN


In [21]:
# filling NA by mean
dataset['Item_Weight'].fillna(dataset['Item_Weight'].mean(), inplace=True)

In [22]:
#Check Missing Values
dataset.apply(lambda x: sum(x.isnull()),axis=0) # axis 0 means column wise. 1 means row wise.

Item_Identifier                 0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
Item_Weight                     0
dtype: int64

In [23]:
# there are 2410 rows which has null values for outlet_size
dataset[dataset['Outlet_Size'].isnull()].head()


,Item_Identifier,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Weight
3,FDX07,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800,19.200
8,FDH17,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986,16.200
9,FDU28,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350,19.200
25,NCD06,Low Fat,0.099887,Household,45.9060,OUT017,2007,NaN,Tier 2,Supermarket Type1,838.9080,13.000
28,FDE51,Regular,0.161467,Dairy,45.5086,OUT010,1998,NaN,Tier 3,Grocery Store,178.4344,5.925


In [24]:
# we can find out outlet_size from outlet_Identifier

In [25]:
dataset[dataset['Outlet_Identifier']=='OUT017'].head()

,Item_Identifier,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Weight
9,FDU28,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350,19.200
25,NCD06,Low Fat,0.099887,Household,45.9060,OUT017,2007,NaN,Tier 2,Supermarket Type1,838.9080,13.000
53,FDA43,Low Fat,0.065042,Fruits and Vegetables,196.3794,OUT017,2007,NaN,Tier 2,Supermarket Type1,3121.2704,10.895
54,NCP18,Low Fat,0.028760,Household,151.4708,OUT017,2007,NaN,Tier 2,Supermarket Type1,4815.0656,12.150
73,FDG02,Low Fat,0.011325,Canned,189.6188,OUT017,2007,NaN,Tier 2,Supermarket Type1,2285.0256,7.855


In [26]:
#dataset.pivot_table(values='Outlet_Size', index='Outlet_Identifier')
from scipy.stats import mode
dataset.pivot_table(values='Outlet_Size', columns='Outlet_Identifier',aggfunc=(lambda x:mode(x).mode[0]) ).T

/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


,Outlet_Size
Outlet_Identifier,
OUT010,0
OUT013,High
OUT017,0
OUT018,Medium
OUT019,Small
OUT027,Medium
OUT035,Small
OUT045,0
OUT046,Small


In [27]:
(dataset['Outlet_Identifier']).unique()

array(['OUT049', 'OUT018', 'OUT010', 'OUT013', 'OUT027', 'OUT045',
       'OUT017', 'OUT046', 'OUT035', 'OUT019'], dtype=object)

In [28]:
dataset[dataset['Outlet_Size'].isnull()]['Outlet_Identifier'].unique()

array(['OUT010', 'OUT045', 'OUT017'], dtype=object)

In [29]:
#Lets impute this later

#dataset = dataset.drop(columns=["Outlet_Size"])

In [30]:
dataset = dataset.drop(columns=["Item_Identifier",'Outlet_Identifier'])

In [31]:
dataset[['Item_Type','Item_Outlet_Sales']].groupby('Item_Type').agg(['mean']) # mean overall 2100
dataset=dataset.drop(columns=['Item_Type'])

In [32]:
dataset[['Outlet_Type','Item_Outlet_Sales']].groupby('Outlet_Type').agg(['median','min','max']) # mean overall 2100
# It seems that there is relation between Outlet_type and Item_Outlet_Sales
# Glocery store has less Item_Outlet_Sales
# Supermarket type 1 and 2 has almost equal Item_Outlet_Sales
# Supermarket 3 has high Item_Outlet_Sales

Item_Outlet_Sales                      
                             median       min         max
Outlet_Type                                              
Grocery Store              256.9988   33.2900   1775.6886
Supermarket Type1         1990.7420   73.2380  10256.6490
Supermarket Type2         1655.1788   69.2432   6768.5228
Supermarket Type3         3364.9532  241.6854  13086.9648

In [33]:
dataset['Outlet_Type']=dataset['Outlet_Type'].str.replace('Grocery Store','2')
dataset['Outlet_Type']=dataset['Outlet_Type'].str.replace('Supermarket Type1','19')
dataset['Outlet_Type']=dataset['Outlet_Type'].str.replace('Supermarket Type2','16')
dataset['Outlet_Type']=dataset['Outlet_Type'].str.replace('Supermarket Type3','34')
dataset['Outlet_Type']=pandas.to_numeric(dataset['Outlet_Type'], errors='coerce')

In [34]:
dataset[['Outlet_Size','Item_Outlet_Sales']].groupby('Outlet_Size').agg(['median','min','max']) # mean overall 2100

Item_Outlet_Sales                     
                       median      min         max
Outlet_Size                                       
High                2050.6640  73.2380  10256.6490
Medium              2251.0698  69.2432  13086.9648
Small               1544.6560  33.9558   9779.9362

In [35]:
dataset['Outlet_Size']=dataset['Outlet_Size'].str.replace('High','20')
dataset['Outlet_Size']=dataset['Outlet_Size'].str.replace('Medium','22')
dataset['Outlet_Size']=dataset['Outlet_Size'].str.replace('Small','15')
dataset['Outlet_Size']=pandas.to_numeric(dataset['Outlet_Size'], errors='coerce')
dataset['Outlet_Size'].fillna(dataset['Outlet_Size'].mean(), inplace=True)

In [36]:
dataset[['Item_Fat_Content','Item_Outlet_Sales']].groupby('Item_Fat_Content').agg(['median','min','max','count']) # mean overall 2100

Item_Outlet_Sales                           
                            median      min         max count
Item_Fat_Content                                             
LF                       1797.6600  40.6138   7503.5660   316
Low Fat                  1765.7016  33.2900  13086.9648  5089
Regular                  1845.5976  33.9558  12117.5600  2889
low fat                  1614.2321  83.8908   9562.2196   112
reg                      1492.7236  37.9506   7833.8028   117

In [37]:
dataset['Item_Fat_Content']=dataset['Item_Fat_Content'].str.replace('LF','17')
dataset['Item_Fat_Content']=dataset['Item_Fat_Content'].str.replace('Low Fat','17')
dataset['Item_Fat_Content']=dataset['Item_Fat_Content'].str.replace('low fat','16')
dataset['Item_Fat_Content']=dataset['Item_Fat_Content'].str.replace('Regular','18')
dataset['Item_Fat_Content']=dataset['Item_Fat_Content'].str.replace('reg','14')
dataset['Item_Fat_Content']=pandas.to_numeric(dataset['Item_Fat_Content'], errors='coerce')

In [38]:
dataset[['Outlet_Location_Type','Item_Outlet_Sales']].groupby('Outlet_Location_Type').agg(['median','min','max','count']) # mean overall 2100

Item_Outlet_Sales                           
                                median      min         max count
Outlet_Location_Type                                             
Tier 1                       1487.3972  33.9558   9779.9362  2388
Tier 2                       2004.0580  99.8700   9664.7528  2785
Tier 3                       1812.3076  33.2900  13086.9648  3350

In [39]:
dataset['Outlet_Location_Type']=dataset['Outlet_Location_Type'].str.replace('Tier 1','1487')
dataset['Outlet_Location_Type']=dataset['Outlet_Location_Type'].str.replace('Tier 2','2004')
dataset['Outlet_Location_Type']=dataset['Outlet_Location_Type'].str.replace('Tier 3','1812')

dataset['Outlet_Location_Type']=pandas.to_numeric(dataset['Outlet_Location_Type'], errors='coerce')

In [40]:
dataset=pandas.get_dummies(dataset, 
                                 prefix=None, prefix_sep='_', 
                                 dummy_na=False, columns=None, sparse=False, drop_first=False, dtype=None)

In [41]:
dataset.head().T

,0,1,2,3,4
Item_Fat_Content,17.000000,18.000000,17.00000,18.000000,17.0000
Item_Visibility,0.016047,0.019278,0.01676,0.000000,0.0000
Item_MRP,249.809200,48.269200,141.61800,182.095000,53.8614
Outlet_Establishment_Year,1999.000000,2009.000000,1999.00000,1998.000000,1987.0000
Outlet_Size,22.000000,22.000000,22.00000,18.960576,20.0000
Outlet_Location_Type,1487.000000,1812.000000,1487.00000,1812.000000,1812.0000
Outlet_Type,19.000000,16.000000,19.00000,2.000000,19.0000
Item_Outlet_Sales,3735.138000,443.422800,2097.27000,732.380000,994.7052
Item_Weight,9.300000,5.920000,17.50000,19.200000,8.9300


# Build Models

In [42]:
model_DF=dataset

In [43]:
from sklearn import model_selection # for splitting into train and test
import sklearn
# Split-out validation dataset
X = model_DF.drop(columns='Item_Outlet_Sales')
Y = model_DF['Item_Outlet_Sales']

validation_size = 0.20
seed = 100
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [44]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [45]:
model_LR = LinearRegression()
model_LR.fit(X_train,Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [46]:
trainResult_LR=model_LR.predict(X_train)
testResult_LR=model_LR.predict(X_test)

In [47]:
def adj_r2_score(model,actual,predicted):
        from sklearn import metrics
        adj = 1 - float(len(actual)-1)/(len(actual)-len(model.coef_)-1)*(1 - metrics.r2_score(actual,predicted))
        return adj
    
    

########## TRAIN DATA RESULT ##########

print('---------- TRAIN DATA RESULT ----------')
# The mean squared error
print("Mean squared error: %.0f"% mean_squared_error(Y_train, trainResult_LR))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.4f' % r2_score(Y_train, trainResult_LR))


print('Adj R2 score: %.4f' % adj_r2_score(model_LR,Y_train, trainResult_LR))

########## TEST DATA RESULT ##########

print('---------- TEST DATA RESULT ----------')
# The mean squared error
print("Mean squared error: %.0f"% mean_squared_error(Y_test, testResult_LR))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.4f' % r2_score(Y_test, testResult_LR))

print('Adj R2 score: %.4f' % adj_r2_score(model_LR,Y_test, testResult_LR))


---------- TRAIN DATA RESULT ----------
Mean squared error: 1262157
Variance score: 0.5650
Adj R2 score: 0.5645
---------- TEST DATA RESULT ----------
Mean squared error: 1332354
Variance score: 0.5484
Adj R2 score: 0.5463


In [48]:
model_RidgeLR = Ridge(alpha=0.05)
model_RidgeLR.fit(X_train,Y_train)

trainResult_RidgeLR=model_RidgeLR.predict(X_train)
testResult_RidgeLR=model_RidgeLR.predict(X_test)

########## TRAIN DATA RESULT ##########

print('---------- TRAIN DATA RESULT ----------')
# The mean squared error
print("Mean squared error: %.0f"% mean_squared_error(Y_train, trainResult_RidgeLR))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.4f' % r2_score(Y_train, trainResult_RidgeLR))


print('Adj R2 score: %.4f' % adj_r2_score(model_RidgeLR,Y_train, trainResult_RidgeLR))


########## TEST DATA RESULT ##########
print('---------- TEST DATA RESULT ----------')
# The mean squared error
print("Mean squared error: %.0f"% mean_squared_error(Y_test, testResult_RidgeLR))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.4f' % r2_score(Y_test, testResult_RidgeLR))

print('Adj R2 score: %.4f' % adj_r2_score(model_RidgeLR,Y_test, testResult_RidgeLR))

---------- TRAIN DATA RESULT ----------
Mean squared error: 1262157
Variance score: 0.5650
Adj R2 score: 0.5645
---------- TEST DATA RESULT ----------
Mean squared error: 1332356
Variance score: 0.5484
Adj R2 score: 0.5463


In [49]:
model_LassoLR = Lasso(alpha=0.05)
model_LassoLR.fit(X_train,Y_train)

trainResult_LassoLR=model_LassoLR.predict(X_train)
testResult_LassoLR=model_LassoLR.predict(X_test)


########## TRAIN DATA RESULT ##########

print('---------- TRAIN DATA RESULT ----------')
# The mean squared error
print("Mean squared error: %.0f"% mean_squared_error(Y_train, trainResult_LassoLR))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.4f' % r2_score(Y_train, trainResult_LassoLR))


print('Adj R2 score: %.4f' % adj_r2_score(model_LassoLR,Y_train, trainResult_LassoLR))


########## TEST DATA RESULT ##########

print('---------- TEST DATA RESULT ----------')
# The mean squared error
print("Mean squared error: %.0f"% mean_squared_error(Y_test, testResult_LassoLR))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.4f' % r2_score(Y_test, testResult_LassoLR))

print('Adj R2 score: %.4f' % adj_r2_score(model_LassoLR,Y_test, testResult_LassoLR))

---------- TRAIN DATA RESULT ----------
Mean squared error: 1262158
Variance score: 0.5650
Adj R2 score: 0.5645
---------- TEST DATA RESULT ----------
Mean squared error: 1332389
Variance score: 0.5484
Adj R2 score: 0.5463


In [50]:
from scipy import stats

X_Columns=model_DF.drop(columns=['Item_Outlet_Sales']).columns # Name of all columns without target column

tempList=['Intercept']
tempList.extend(X_Columns)

params = numpy.append(model_LR.intercept_,model_LR.coef_)
predictions = model_LR.predict(X_train)

newX = pandas.DataFrame({"Constant":numpy.ones(len(X))}).join(pandas.DataFrame(X))
MSE = mean_squared_error(Y_test, testResult_LR)

var_b = MSE*(numpy.linalg.inv(numpy.dot(newX.T,newX)).diagonal())
sd_b = numpy.sqrt(var_b)
ts_b = params/ sd_b

p_values =[2*(1-stats.t.cdf(numpy.abs(i),(len(newX)-1))) for i in ts_b]

sd_b = numpy.round(sd_b,3)
ts_b = numpy.round(ts_b,3)
p_values = numpy.round(p_values,12)
params = numpy.round(params,4)

myDF3 = pandas.DataFrame()
myDF3["Cols"],myDF3["Coefficients"],myDF3["Standard Errors"],myDF3["t values"],myDF3["Probabilites"] = [tempList,params,sd_b,ts_b,p_values]
display(myDF3)

,Cols,Coefficients,Standard Errors,t values,Probabilites
0,Intercept,-17158.6109,3373.100,-5.087,3.717310e-07
1,Item_Fat_Content,47.2266,19.928,2.370,1.781646e-02
2,Item_Visibility,-304.5119,251.306,-1.212,2.256549e-01
3,Item_MRP,15.5627,0.201,77.473,0.000000e+00
4,Outlet_Establishment_Year,7.3235,1.702,4.303,1.701341e-05
5,Outlet_Size,-5.0041,4.933,-1.014,3.103838e-01
6,Outlet_Location_Type,-0.0808,0.070,-1.149,2.504724e-01
7,Outlet_Type,107.2892,1.880,57.081,0.000000e+00
8,Item_Weight,0.5288,2.694,0.196,8.443754e-01
